In [ ]:
print("starting 4_save_synthetic_reference_images")

This notebook generates and saves the synthetic reference images. 


Large parts of this notebook are taken from https://github.com/bethgelab/testing_visualizations/blob/master/generate_stimuli/save_optimized_stimuli.py on 18.04.2021.

# Imports

In [ ]:
import os

import tensorflow as tf

import pandas as pd
from tqdm import tqdm
import argparse
import time
import numpy as np
import PIL
import argparse

# lucid imports
import lucid.modelzoo.vision_models as models
from lucid.optvis import transform
import lucid.optvis.param as param

# just modified render & objectives file and not the original files from lucid
import render
import objectives

# custom imports
import occlusion_utils as ut

# Load model

In [ ]:
# import InceptionV1 from the Lucid modelzoo
model = models.InceptionV1()
model.load_graphdef()

# Parameters

In [ ]:
tf.set_random_seed(1234)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-s", "--stimuli-dir", required=True, help="Path to save stimuli to.")
parser.add_argument("-t", "--trial-type", required=True, help="instruction_practice_catch or sampled_trials.")
args = parser.parse_args()
print(args)

In [ ]:
stimuli_dir = args.stimuli_dir
trial_type = args.trial_type
_, _, batch_size_optimized = ut.get_number_of_stimuli(stimuli_dir)

# Load experiment specification

In [ ]:
# read in unit specifications from csv into pandas dataframe
path_to_csv_file = os.path.join(stimuli_dir, f"layer_folder_mapping_{trial_type}.csv")
unit_specs_df = pd.read_csv(path_to_csv_file, header=1)

In [ ]:
unit_specs_df

# Functions

In [ ]:
def save_optimized_images_and_objectives(
    max_or_min, images, objectives, loss_additional_global, destination_dir
):
    """save the maximal optimized images"""
    
    # iterate over min and max
    number_images = len(images)
    for img_idx in range(number_images):
        img = PIL.Image.fromarray(np.uint8(np.clip(images[img_idx] * 255, 0, 255)))
        image_name = f"reference_{max_or_min}_{img_idx}.png"
        image_path = os.path.join(destination_dir, image_name)
        print(image_path)
        img.save(image_path)

    np.save(
        os.path.join(destination_dir, f"{max_or_min}_objective_values.npy"),
        objectives,
    )
    np.save(
        os.path.join(
            destination_dir,
            f"{max_or_min}_additional_global_diversity_loss.npy",
        ),
        loss_additional_global,
    )

In [ ]:
def get_channel_objective_stimuli(layer, feature_map):
    img_size = 224

    padding_size = 16
    param_f = lambda: param.image(img_size + 2 * padding_size, batch=batch_size_optimized)
    objective_per_image = objectives.channel(layer, feature_map)
    diversity_loss = -1e2 * objectives.diversity(layer)

    # transformations as described in Feature Visualization blog post
    kwargs = dict(
        thresholds=(2560,),
        optimizer=tf.train.AdamOptimizer(learning_rate=0.05),
        transforms=[
            transform.jitter(16),
            transform.random_scale((1.0, 0.975, 1.025, 0.95, 1.05)),
            transform.random_rotate((-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5)),
            transform.jitter(8),
        ],
    )

    # generate max stimuli
    _, max_stimuli, max_loss, loss_additional_global_list = render.render_vis(
        model,
        objective_per_image,
        diversity_loss,
        param_f,
        use_fixed_seed=True,
        **kwargs,
    )
    # the optimization may save multiple states of the results
    # the last item is the final value
    max_stimuli = max_stimuli[-1]
    max_loss = max_loss[-1]
    max_loss_additional_global = loss_additional_global_list[-1]

    # undo/crop padding
    max_stimuli = max_stimuli[:, padding_size:-padding_size, padding_size:-padding_size]

    return (
        max_stimuli,
        max_loss,
        max_loss_additional_global,
    )

# Generate the synthetic reference images

In [ ]:
# loop through each row
for idx, cur_row in tqdm(unit_specs_df.iterrows(), total=len(unit_specs_df)):
    start_time = time.time()
    layer = f"{cur_row['layer_name']}_{cur_row['pre_post_relu']}"
    feature_map = cur_row["feature_map_number"]

    (
        max_stimuli,
        max_loss,
        max_loss_additional_global,
    ) = get_channel_objective_stimuli(layer, feature_map)

    # save images
    destination_dir = os.path.join(
            stimuli_dir,
            ut.objective,
            trial_type,
            f"layer_{cur_row['layer_number']}",
            f"kernel_size_{cur_row['kernel_size_number']}",
            f"channel_{cur_row['channel_number']}",
            "optimized_images",
        )
    os.makedirs(destination_dir, exist_ok=True)
    save_optimized_images_and_objectives(
        "max", max_stimuli, max_loss, max_loss_additional_global, destination_dir
    )
    print(f"layer: {layer}, feature_map: {feature_map}, time = {time.time() - start_time:.1f}s")

In [ ]:
print("done with 4_save_synthetic_reference_images")